# 1) Generating Grid Points Within Shapefile Boundaries

    This code walks through a specified input folder to find shapefiles (.shp), then generates and saves a grid of points spaced by a given pixel size within the boundaries of each shapefile. It sets the coordinate reference system (CRS) for each shapefile, calculates the number of points to fit within the shapefile's bounding box based on the pixel size, and filters out points that do not fall within the shapefile's geometries before saving them to a specified output folder.

In [ ]:
import os
import geopandas as gpd
from shapely.geometry import Point

input_folder = '/path/to/tiles/'

output_folder = '/path/to/save/grid_points'

pixel_size = 20 

for root, dirs, files in os.walk(input_folder):
    for file in files:
        if file.endswith(".shp"):
            shp_path = os.path.join(root, file)
            gdf = gpd.read_file(shp_path)

            gdf.crs = f"EPSG:32631"

            bbox = gdf.bounds.iloc[0]

            num_points_x = int((bbox.maxx - bbox.minx) / pixel_size) + 1
            num_points_y = int((bbox.maxy - bbox.miny) / pixel_size) + 1

            points = []
            for i in range(num_points_x):
                for j in range(num_points_y):
                    x = bbox.minx + i * pixel_size + 0.5 * pixel_size
                    y = bbox.miny + j * pixel_size + 0.5 * pixel_size
                    point = Point(x, y)
                    if gdf.geometry.contains(point).any():
                        points.append(point)

            points_gdf = gpd.GeoDataFrame(geometry=points, crs=gdf.crs)

            output_path = os.path.join(output_folder, f"{file}")
            points_gdf.to_file(output_path)
            
print("Process completed")